In [33]:
import sys
sys.path.append('D:\PyCharmProjects\VFPUMC02')
sys.path.append(r'C:\Users\Administrator\PycharmProjects\VFPUMC02')
sys.path.append(r'/root/VFPUMC02')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
from datasets.DataSet import BankDataset
from utils.DataProcessUtils import split_labeled_unlabeled, vertical_split_array
from utils.DataProcessUtils import find_rounds_math
from utils.FateUtils import evaluate_model
from classfiers.VF_TwoStep import VF_TwoStep
from vf4lr.VF_LR import VF_LR
from classfiers.VF_LinearRegression import VF_LinearRegression
from enums.ModelType import ModelType

In [35]:
bankDataset = BankDataset()
X, y = bankDataset.get_data()
X, y_C, y_R = X[:,:-1], y, X[:,-1]
XA, XB = vertical_split_array(X,random_state=24,first_col_rate=0.5)

2025-01-23 22:21:46,148 - INFO - 正在加载数据集 bank.csv
2025-01-23 22:21:46,353 - INFO - 数据集 bank.csv 加载完成，数据集形状为 (41188, 64)


切分后XA的形状为(41188, 31)
切分后XB的形状为(41188, 31)


In [36]:
labeled, unlabeled = split_labeled_unlabeled(XA,XB,y_C,y_R,random_state=24,k=0.2)

总样本数: 41188
输入数组 0 的形状: (41188, 31)
输入数组 1 的形状: (41188, 31)
输入数组 2 的形状: (41188,)
输入数组 3 的形状: (41188,)
选择 8237 个样本作为有标签数据，剩余样本作为无标签数据
输入数组 0 的有标签数据形状: (8237, 31)
输入数组 0 的无标签数据形状: (32951, 31)
输入数组 1 的有标签数据形状: (8237, 31)
输入数组 1 的无标签数据形状: (32951, 31)
输入数组 2 的有标签数据形状: (8237,)
输入数组 2 的无标签数据形状: (32951,)
输入数组 3 的有标签数据形状: (8237,)
输入数组 3 的无标签数据形状: (32951,)


In [37]:
# 解包 labeled（有标签数据）
XA_L, XB_L, y_C_L, y_R_L = labeled
# 解包 unlabeled（无标签数据
XA_U, XB_U, y_C_U, y_R_U = unlabeled

In [38]:
vf_clf = VF_LR(
    learning_rate=0.2,
    epoch_num=5,
    batch_size=128
)
vf_reg = VF_LinearRegression(
    config={
    'batch_size': 32,
    'lr': 0.01,
    'lambda': 0.1,
    'n_iters':5
    }
)

2025-01-23 22:21:46,483 - ConsoleLogger - INFO - VF_LR 模型初始化完成。
2025-01-23 22:21:46,483 - INFO - VF_LR 模型初始化完成。
2025-01-23 22:21:46,484 - ConsoleLogger - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 128, 'client_num': 2}
2025-01-23 22:21:46,484 - INFO - 模型配置: {'learning_rate': 0.2, 'epoch_num': 5, 'batch_size': 128, 'client_num': 2}
2025-01-23 22:21:46,485 - ConsoleLogger - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 22:21:46,485 - INFO - VF_LinearRegression 模型初始化完成。
2025-01-23 22:21:46,486 - ConsoleLogger - INFO - 模型配置: {'batch_size': 32, 'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 22:21:46,486 - INFO - 模型配置: {'batch_size': 32, 'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}


In [39]:
vf_two_step = VF_TwoStep(
    clf=vf_clf,
    reg=vf_reg,
    k=0.25,
    max_iter=5
)

2025-01-23 22:21:46,514 - INFO - VF_TwoStep 类已成功初始化: max_iter=5, k=0.25, min_confidence=0.00, convergence_threshold=0, clf=VF_LR, reg=VF_LinearRegression


In [40]:
vf_two_step.fit(
    XA_L=XA_L,
    XB_L=XB_L,
    y_L=y_C_L,
    XA_U=XA_U,
    XB_U=XB_U
)

2025-01-23 22:21:46,542 - INFO - 有标签数据 (L): 8237 样本
2025-01-23 22:21:46,543 - INFO - 无标签数据 (U): 32951 样本
2025-01-23 22:21:46,544 - INFO - 任务类型判定结果: classification
2025-01-23 22:21:46,545 - INFO - 这是一个分类任务。
2025-01-23 22:21:46,546 - INFO - 分类任务中的类别: [0 1]
2025-01-23 22:21:46,546 - INFO - [INIT] 初始化 self.pred, 未标注数据数量=32951
2025-01-23 22:21:46,547 - INFO - [INIT] 最大迭代次数 max_iter=5, 每轮选取比例 k=0.25
2025-01-23 22:21:46,547 - INFO - ===== [Epoch 1/5] 迭代开始 =====
2025-01-23 22:21:46,548 - INFO - [INFO] 当前 Labeled 样本数量: 8237, Unlabeled 样本数量: 32951
2025-01-23 22:21:46,548 - INFO - [STEP 1] 开始训练分类器 (基于 8237 个 Labeled 样本)...
2025-01-23 22:21:46,550 - ConsoleLogger - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (8237, 31), 客户端B训练数据形状: (8237, 31), 标签大小: 8237
2025-01-23 22:21:46,550 - INFO - 开始训练垂直联邦逻辑回归模型，客户端A训练数据形状: (8237, 31), 客户端B训练数据形状: (8237, 31), 标签大小: 8237
2025-01-23 22:21:46,551 - ConsoleLogger - INFO - 检测到的类别数量: 2
2025-01-23 22:21:46,551 - INFO - 检测到的类别数量: 2
2025-01-23 22:21:46,552 - ConsoleLogger - IN

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 22:21:46,666 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 22:21:46,666 - INFO - 第 1 轮训练完成。
2025-01-23 22:21:46,668 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:46,668 - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:46,768 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 22:21:46,768 - INFO - 第 2 轮训练完成。
2025-01-23 22:21:46,769 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:46,769 - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:46,866 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 22:21:46,866 - INFO - 第 3 轮训练完成。
2025-01-23 22:21:46,867 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:46,867 - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:46,965 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 22:21:46,965 - INFO - 第 4 轮训练完成。
2025-01-23 22:21:46,966 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:46,966 - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:47,063 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 22:21:47,063 - INFO - 第 5 轮训练完成。
2025-01-23 22:21:47,065 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 22:21:48,568 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 22:21:48,568 - INFO - 第 1 轮训练完成。
2025-01-23 22:21:48,569 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:48,569 - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:48,765 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 22:21:48,765 - INFO - 第 2 轮训练完成。
2025-01-23 22:21:48,766 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:48,766 - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:48,959 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 22:21:48,959 - INFO - 第 3 轮训练完成。
2025-01-23 22:21:48,961 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:48,961 - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:49,155 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 22:21:49,155 - INFO - 第 4 轮训练完成。
2025-01-23 22:21:49,156 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:49,156 - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:49,350 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 22:21:49,350 - INFO - 第 5 轮训练完成。
2025-01-23 22:21:49,351 - Consol

...

2025-01-23 22:21:51,217 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 22:21:51,217 - INFO - 第 1 轮训练完成。
2025-01-23 22:21:51,218 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:51,218 - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:51,486 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 22:21:51,486 - INFO - 第 2 轮训练完成。
2025-01-23 22:21:51,488 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:51,488 - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:51,758 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 22:21:51,758 - INFO - 第 3 轮训练完成。
2025-01-23 22:21:51,759 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:51,759 - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:52,028 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 22:21:52,028 - INFO - 第 4 轮训练完成。
2025-01-23 22:21:52,029 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:52,029 - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:52,301 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 22:21:52,301 - INFO - 第 5 轮训练完成。
2025-01-23 22:21:52,302 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 22:21:54,795 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 22:21:54,795 - INFO - 第 1 轮训练完成。
2025-01-23 22:21:54,796 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:54,796 - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:55,205 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 22:21:55,205 - INFO - 第 2 轮训练完成。
2025-01-23 22:21:55,206 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:55,206 - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:55,602 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 22:21:55,602 - INFO - 第 3 轮训练完成。
2025-01-23 22:21:55,603 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:55,603 - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:56,003 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 22:21:56,003 - INFO - 第 4 轮训练完成。
2025-01-23 22:21:56,004 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:56,004 - INFO - 开始第 5/5 轮训练...
2025-01-23 22:21:56,387 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 22:21:56,387 - INFO - 第 5 轮训练完成。
2025-01-23 22:21:56,389 - Consol

检测到二分类任务 (Binary Classification)
输入标签 y 的唯一值个数: 2, 唯一值: [0 1]


2025-01-23 22:21:59,166 - ConsoleLogger - INFO - 第 1 轮训练完成。
2025-01-23 22:21:59,166 - INFO - 第 1 轮训练完成。
2025-01-23 22:21:59,167 - ConsoleLogger - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:59,167 - INFO - 开始第 2/5 轮训练...
2025-01-23 22:21:59,540 - ConsoleLogger - INFO - 第 2 轮训练完成。
2025-01-23 22:21:59,540 - INFO - 第 2 轮训练完成。
2025-01-23 22:21:59,542 - ConsoleLogger - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:59,542 - INFO - 开始第 3/5 轮训练...
2025-01-23 22:21:59,904 - ConsoleLogger - INFO - 第 3 轮训练完成。
2025-01-23 22:21:59,904 - INFO - 第 3 轮训练完成。
2025-01-23 22:21:59,905 - ConsoleLogger - INFO - 开始第 4/5 轮训练...
2025-01-23 22:21:59,905 - INFO - 开始第 4/5 轮训练...
2025-01-23 22:22:00,274 - ConsoleLogger - INFO - 第 4 轮训练完成。
2025-01-23 22:22:00,274 - INFO - 第 4 轮训练完成。
2025-01-23 22:22:00,275 - ConsoleLogger - INFO - 开始第 5/5 轮训练...
2025-01-23 22:22:00,275 - INFO - 开始第 5/5 轮训练...
2025-01-23 22:22:00,636 - ConsoleLogger - INFO - 第 5 轮训练完成。
2025-01-23 22:22:00,636 - INFO - 第 5 轮训练完成。
2025-01-23 22:22:00,638 - Consol

In [41]:
y_C_pred = vf_two_step.get_unlabeled_predict(ModelType.CLASSIFICATION)

In [42]:
evaluate_model(y_C_U,y_C_pred)


========== Classification Metrics ==========
Accuracy         : 0.1121
Precision (macro): 0.5559
Recall (macro)   : 0.5002
F1 (macro)       : 0.1009

Confusion Matrix:
[[   11 29258]
 [    0  3682]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     29269
           1       0.11      1.00      0.20      3682

    accuracy                           0.11     32951
   macro avg       0.56      0.50      0.10     32951
weighted avg       0.90      0.11      0.02     32951



In [43]:
vf_two_step.fit(
    XA_L=XA_L,
    XB_L=XB_L,
    y_L=y_R_L,
    XA_U=XA_U,
    XB_U=XB_U
)

2025-01-23 22:22:04,885 - INFO - 有标签数据 (L): 8237 样本
2025-01-23 22:22:04,886 - INFO - 无标签数据 (U): 32951 样本
2025-01-23 22:22:04,887 - INFO - 任务类型判定结果: regression
2025-01-23 22:22:04,887 - INFO - 这是一个回归任务。
2025-01-23 22:22:04,888 - INFO - 回归任务标签统计: 最小值=0.0000, 最大值=1.0000, 平均值=0.0002, 标准差=0.0156
2025-01-23 22:22:04,889 - INFO - [INIT] 初始化 self.pred, 未标注数据数量=32951
2025-01-23 22:22:04,889 - INFO - [INIT] 最大迭代次数 max_iter=5, 每轮选取比例 k=0.25
2025-01-23 22:22:04,889 - INFO - ===== [Epoch 1/5] 迭代开始 =====
2025-01-23 22:22:04,890 - INFO - [INFO] 当前 Labeled 样本数量: 8237, Unlabeled 样本数量: 32951
2025-01-23 22:22:04,890 - INFO - [STEP 1] 开始训练回归器 (基于 8237 个 Labeled 样本)...
2025-01-23 22:22:04,891 - ConsoleLogger - INFO - 开始构建A、B、C三个客户端...
2025-01-23 22:22:04,891 - INFO - 开始构建A、B、C三个客户端...
2025-01-23 22:22:04,892 - ConsoleLogger - INFO - Client 初始化完成，config: {'batch_size': 32, 'lr': 0.01, 'lambda': 0.1, 'n_iters': 5}
2025-01-23 22:22:04,892 - INFO - Client 初始化完成，config: {'batch_size': 32, 'lr': 0.01, 'lambda': 

In [44]:
y_R_pred = vf_two_step.get_unlabeled_predict(ModelType.REGRESSION)

In [45]:
evaluate_model(y_R_U, y_R_pred, model_type=ModelType.REGRESSION)


========== Regression Metrics ==========
MSE               : 0.0005
RMSE              : 0.0220
MAE               : 0.0005
R^2 (Coefficient) : -0.0004
Explained Variance: -0.0000
